In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('/content/fashion-mnist_train.csv')

In [ ]:
print(torch.cuda.is_available())

False


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [ ]:
torch.manual_seed(42)

In [ ]:
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,9,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6,0,0,0,0,0,0,0,5,0,...,0.0,0.0,0.0,30.0,43.0,0.0,0.0,0.0,0.0,0.0
3,0,0,0,0,1,2,0,0,0,0,...,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,3,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df.shape

(60000, 785)

In [ ]:
x = df.iloc[:,1:]
y = df.iloc[:,0]

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
#trandformations
from torchvision.transforms import transforms

transform  = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.48235, 0.45882, 0.40784],std = [0.00392156862745098, 0.00392156862745098, 0.00392156862745098])
])


In [ ]:
x_train.shape[1]

784

In [ ]:
from PIL import Image
import numpy as np
#create coustomdataset class
class CustomDataset(Dataset):
  def __init__(self,features,labels,transform):
    self.features = features
    self.transform = transform
    self.labels = labels

  def __len__(self):
    return len(self.features)

  def __getitem__(self,index):

    #reshape
    image = self.features.iloc[index].values.reshape(28,28)

    #convert data type
    image = image.astype(np.uint8)

    #convert into rgb
    #convert from (c,h,w) to (h,w,c) by using axis -1
    image = np.stack([image]*3,axis = -1)

    #convert to into PIL image
    image  = Image.fromarray(image)

    #apply transformations
    image  = self.transform(image)

    return image, torch.tensor(self.labels.iloc[index],dtype = torch.long)

In [ ]:
test_dataset = CustomDataset(x_test,y_test,transform)

In [ ]:
train_dataset = CustomDataset(x_train,y_train,transform)

In [ ]:
import torchvision.models as models
vgg16 = models.vgg16(pretrained = True)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
vgg16.features

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_si

In [ ]:
vgg16.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [ ]:
for parm in vgg16.features.parameters():
  parm.requires_grad =False

In [ ]:
vgg16.classifier = nn.Sequential(
    nn.Linear(25088,1024),
    nn.ReLU(),
    nn.Linear(1024,512),
    nn.ReLU(),
    nn.Linear(512,10)
)

In [ ]:
# #nn class
# class NeuralNetwork(nn.Module):
#   def __init__(self,input_dim):
#     super().__init__()
#     self.features = nn.Sequential(
#         nn.Conv2d(input_dim, 32,kernel_size=3,padding="same"),
#         nn.ReLU(),
#         nn.BatchNorm2d(32),
#         nn.MaxPool2d(kernel_size = 2, stride = 2),

#         nn.Conv2d(32,64,kernel_size=3,padding="same"),
#         nn.ReLU(),
#         nn.BatchNorm2d(64),
#         nn.MaxPool2d(kernel_size = 2, stride = 2)
#     )
#     self.classifier = nn.Sequential(
#         nn.Flatten(),
#         nn.Linear(3136,128),
#         nn.ReLU(),
#         nn.Dropout(0.3),

#         nn.Linear(128,64),
#         nn.ReLU(),
#         nn.Dropout(0.3),

#         nn.Linear(64,10)
#         )


#   def forward(self,x):
#     x = self.features(x)
#     x = self.classifier(x)
#     return x

In [ ]:
learning_rate = 0.001
epochs = 10
batches = 32

In [ ]:
test_loader = DataLoader(test_dataset,batch_size=batches,shuffle=False)
train_loader = DataLoader(train_dataset,batch_size=batches,shuffle=True)

In [ ]:
#creating a model

vgg16 = vgg16.to(device)

#loss function
loss = nn.CrossEntropyLoss()

#optimizer
optimizer = torch.optim.SGD(vgg16.classifier.parameters(),lr=learning_rate )

In [ ]:
#training loop
for epoch in range(epochs) :
  epoch_loss = 0.0
  for batch_fertures,batch_labels in train_loader:

    #moving to gpu
    batch_fertures = batch_fertures.to(device)
    batch_labels = batch_labels.to(device)

    #forward pass
    y_pred = vgg16(batch_fertures)

    #loss calculation
    loss_value = loss(y_pred,batch_labels)

    #making gradindients zero
    optimizer.zero_grad()

    #backward pass
    loss_value.backward()

    #updating the weights and bias
    optimizer.step()
    epoch_loss += loss_value.item()
  avg = epoch_loss/len(train_loader)
  print(f"epochs = {epoch+1} and loss = {avg}")

In [ ]:
#evaluting on train data

total = 0
correct = 0

for batch_features,batch_labels in train_loader:
  #moving the parameter on gpu
  batch_features = batch_features.to(device)
  batch_labels = batch_labels.to(device)

  #forward pass
  y_pred = model(batch_features)

  #finding values
  _,predicted = torch.max(y_pred,1)
  total += batch_labels.size(0)
  correct += (predicted == batch_labels).sum().item()

accuracy = correct/total
print(f"accuracy = {accuracy}")

accuracy = 0.99825


In [ ]:
#evaluating on test data
total = 0
correct = 0

for batch_features, batch_labels in test_loader:
  batch_features = batch_features.to(device)
  batch_labels = batch_labels.to(device)

  y_pred=model(batch_features)

  _,predicted = torch.max(y_pred,1)
  total +=batch_labels.size(0)
  correct +=(predicted == batch_labels).sum().item()

accuracy = correct/total
print(f"accuracy = {accuracy}")

accuracy = 0.9156666666666666
